In [1]:
from tkinter import *
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
from tkinter import messagebox
import mysql.connector
from datetime import datetime
from time import strftime
import cv2
import os
import numpy as np

class face_recognition:
    def __init__(self,root):
        self.root=root
        self.root.geometry("1530x790+0+0")
        self.root.resizable(False,False)
        self.root.title("Face Recognition System")
        title_lbl = Label(self.root, text="FACE RECOGNITION", font=("times new roman", 35, "bold"), bg="white", fg="green")
        title_lbl.place(x=0, y=0, width=1600, height=45)
        
        img_left = Image.open("class.png")
        img_left=img_left.resize((650,770),Image.ANTIALIAS)
        self.photoimg_left = ImageTk.PhotoImage(img_left)
        
        f_lbl = Label(self.root, image=self.photoimg_left)
        f_lbl.place(x=0, y=45, width=650, height=770)
        
        img_right = Image.open("attendance.png")
        img_right=img_right.resize((950,770),Image.ANTIALIAS)
        self.photoimg_right = ImageTk.PhotoImage(img_right)
        
        f_lbl = Label(self.root, image=self.photoimg_right)
        f_lbl.place(x=655, y=45, width=950, height=770)
        
        b2 = Button(f_lbl,text="RECOGNIZE",cursor="hand2", font=("times new roman",20 , "bold"),bg="green",fg="white",command=self.face_recog)
        b2.place(x=470,y=490,width=200,height=50)
     
    def mark_attendance(self,i,r,n,d):
        with open("attendance.csv","r+",newline="") as f:
            myDataList=f.readlines()
            name_list=[]
            for line in myDataList:
                entry=line.split((","))
                name_list.append(entry[0])
            if((i not in name_list) and (r not in name_list) and (n not in name_list) and (d not in name_list)):
                now=datetime.now()
                d1=now.strftime("%d %m %y")
                dtString=now.strftime("%H:%M:%S")
                f.writelines(f"{i},{r},{n}, {d}, {dtString}, {d1},Present\n".strip())

 
    def face_recog(self):  
        def draw_boundary(img,classifier,scaleFactor ,minNeighbors, color,text,clf):
            gray_image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            features=classifier.detectMultiScale(gray_image,scaleFactor,minNeighbors)
            coord=[]
            for (x,y,w,h) in features:

                cv2.rectangle(img,(x,y), (x+w, y+h), (0,255,0),3)
                id, predict=clf.predict(gray_image[y:y+h,x:x+w])
                confidence=int((100*(1-predict/300)))
                
                conn = mysql.connector.connect(host="localhost", username="root", password="root",database="face_recognizer")
                my_cursor = conn.cursor()

                my_cursor.execute("select Student_id from student where Student_id="+str(id))
                i=my_cursor.fetchone()
                i="+".join(i)

                
                my_cursor.execute("select Name from student where Student_id="+str(id))
                n=my_cursor.fetchone()
                n="+".join(n)


                my_cursor.execute("select Roll from student  where Student_id="+str(id))
                r=my_cursor.fetchone()
                r="+".join(r)

                my_cursor.execute("select Dep from student where Student_id="+str(id))
                d=my_cursor.fetchone()
                d="+".join(d)
                
                
                if confidence>77:
                    cv2.putText (img, f"ID:{i}", (x,y-75),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),3)
                    cv2.putText (img, f"Roll:{r}", (x,y-55),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),3)
                    cv2.putText (img, f"Name:{n}", (x,y-30),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),3)
                    cv2.putText (img, f"Department:{d}", (x,y-5),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),3)
                    self.mark_attendance(i,r,n,d)
                else:
                    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
                    cv2.putText (img, "Unknown", (x,y-5),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),3)
                coord=[x,y,w,h]
            return coord
    
        def recognize(img,clf,faceCascade):
            coord=draw_boundary(img, faceCascade,1.1,10,(255,25,255), "Face",clf)
            return img

        faceCascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        clf=cv2.face.LBPHFaceRecognizer_create()
        clf.read("classifier.xml")

        video_cap=cv2.VideoCapture(0)

        while True:
            ret, img=video_cap.read()
            img=recognize(img,clf,faceCascade)
            cv2.imshow("Welcome TO face Recognition", img)
            if cv2.waitKey(1)==13:
                break

        
        cv2.destroyAllWindows()
        video_cap.release()




if __name__== "__main__":
    root=Tk()
    obj=face_recognition(root)
    root.mainloop()

C:\Users\pal28\AppData\Local\Temp\ipykernel_16956\2225087404.py:23: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_left=img_left.resize((650,770),Image.ANTIALIAS)
C:\Users\pal28\AppData\Local\Temp\ipykernel_16956\2225087404.py:30: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_right=img_right.resize((950,770),Image.ANTIALIAS)
